In [34]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import time
from airline import Airline

In [2]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(r"D:\project\crawler\chromedriver.exe", options=options)

driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})
driver.execute_cdp_cmd("Network.enable", {})
driver.execute_cdp_cmd("Network.setExtraHTTPHeaders", {"headers": {"User-Agent": "browser1"}})

driver.get("http://www.juneyaoair.com/pages/Flight/flight.aspx?flightType=OW&sendCity=上海&sendCode=SHA&arrCity=成都&arrCode=CTU&directType=N&tripType=D&departureDate=2022-08-10&returnDate=2022-08-10")


In [3]:
table = driver.find_element_by_xpath('//*[@id="flt_hd_table"]/table/tbody')
# /html/body/div[1]/div[5]/div[1]/div[1]/div[2]/table/tbody/tr[12]/td/div/a
t = time.time()
clickables = driver.find_elements_by_xpath("/html/body/div[1]/div[5]/div[1]/div[1]/div[2]/table/tbody/tr[@class='flt_more']/td/div/a")
# clickables = driver.find_elements(By.CLASS_NAME, "moreCabin")
for clickable in clickables:
    ActionChains(driver).move_to_element(clickable).click().perform()
print(">>>耗时:", time.time()-t)

>>>耗时: 1.4779047966003418


airline, al_num, al_type,al_size, departure_time, arrival_time,departure_place, arrival_place,promotion, price,departure_rate, arrival_rate,delay_time, advance_time,devices, cabin))

In [39]:
airline = None
for tr in table.find_elements_by_xpath("./tr"):

    if tr.get_attribute("class") == "title":
        if airline is not None:
            print("\t".join(["航空公司", "航班号", "机型", "机型大小", "出发时间", "到达时间", "出发地", "目的地", "优惠",
                         "价格", "到达准点率", "出发准点率", "平均延误", "平均提前", "服务", "舱位"]))
            print("\t".join(airline.features()))
        al_num = tr.find_elements_by_class_name("flt_No")[0].text
        al_type = tr.find_elements_by_class_name("flt_showTypeInfo")[0].text
        al_type, al_size = al_type.strip("）").split("（")
        t = tr.find_elements_by_class_name("flt_date")
        p = tr.find_elements_by_class_name("flt_site")
        departure_time = t[0].text
        arrival_time = t[1].text
        departure_place= p[0].text
        arrival_place = p[1].text
        airline = Airline(al_num, "吉祥航空", al_type, al_size, departure_time, arrival_time, departure_place, arrival_place)
    elif tr.get_attribute("class") == "cnt":
        cabin = tr.find_element_by_xpath("./td[1]").text
        cabin = cabin.split(' ')[0]
        cabin, promotion = cabin.strip(")").split("(")
        devices = tr.find_element_by_xpath('./td[3]/span').text
        price = tr.find_element_by_xpath('./td[7]/div/div/span').text
        promotion += ("," + tr.find_element_by_xpath('./td[9]').text)
        airline.cabin = cabin
        airline.price = price
        airline.promotion = promotion
        airline.devices = devices


航空公司	航班号	机型	机型大小	出发时间	到达时间	出发地	目的地	优惠	价格	到达准点率	出发准点率	平均延误	平均提前	服务	舱位
吉祥航空	HO1269	全新空客320-neo	中型 	06:50	12:05	浦东T2	天府T2	Q,7.1折	1808					含点心	经济舱
航空公司	航班号	机型	机型大小	出发时间	到达时间	出发地	目的地	优惠	价格	到达准点率	出发准点率	平均延误	平均提前	服务	舱位
吉祥航空	HO1119	波音787-9	大型	08:05	11:15	虹桥T2	天府T2	V,6.0折	1548					含早餐	经济舱
航空公司	航班号	机型	机型大小	出发时间	到达时间	出发地	目的地	优惠	价格	到达准点率	出发准点率	平均延误	平均提前	服务	舱位
吉祥航空	HO1097	空客321	中型	12:15	15:25	虹桥T2	天府T2	V,6.0折	1548					含点心	经济舱
航空公司	航班号	机型	机型大小	出发时间	到达时间	出发地	目的地	优惠	价格	到达准点率	出发准点率	平均延误	平均提前	服务	舱位
吉祥航空	HO1059	全新空客321-neo	中型	15:05	18:15	虹桥T2	天府T2	V,6.0折	1548					含点心	经济舱


In [43]:
cabin, promotion = "经济舱(T)".strip(")").split("(")


In [45]:
import jieba
string = '在授课的同时，这个“讲台”正以约23倍于音速的第一宇宙速度环绕地球飞行，舱外是真空、200多摄氏度的温差、充满宇宙射线的极端环境。'
result = jieba.lcut(string)
print(len(result), '/'.join(result))

37 在/授课/的/同时/，/这个/“/讲台/”/正以/约/23/倍/于/音速/的/第一/宇宙速度/环绕/地球/飞行/，/舱外/是/真空/、/200/多摄氏度/的/温差/、/充满/宇宙射线/的/极端/环境/。


In [46]:
import jieba
string = '在授课的同时，这个“讲台”正以约23倍于音速的第一宇宙速度环绕地球飞行，舱外是真空、200多摄氏度的温差、充满宇宙射线的极端环境。'
result = jieba.lcut(string, cut_all=True)
print(len(result), '/'.join(result))

45 在/授课/的/同时/，/这个/“/讲台/”/正/以/约/23/倍/于/音速/的/第一/宇宙/宇宙速度/速度/环绕/地球/飞行/，/舱/外/是/真空/、/200/多摄氏度/摄氏/摄氏度/的/温差/、/充满/宇宙/宇宙射线/射线/的/极端/环境/。


In [47]:
import jieba
string = '在授课的同时，这个“讲台”正以约23倍于音速的第一宇宙速度环绕地球飞行，舱外是真空、200多摄氏度的温差、充满宇宙射线的极端环境。'
result = jieba.lcut_for_search(string)
print(len(result), '/'.join(result))

43 在/授课/的/同时/，/这个/“/讲台/”/正以/约/23/倍/于/音速/的/第一/宇宙/速度/宇宙速度/环绕/地球/飞行/，/舱外/是/真空/、/200/摄氏/摄氏度/多摄氏度/的/温差/、/充满/宇宙/射线/宇宙射线/的/极端/环境/。


In [48]:
jieba.add_word('极端环境')
result = jieba.lcut(string)
print(len(result), '/'.join(result))

36 在/授课/的/同时/，/这个/“/讲台/”/正以/约/23/倍/于/音速/的/第一/宇宙速度/环绕/地球/飞行/，/舱外/是/真空/、/200/多摄氏度/的/温差/、/充满/宇宙射线/的/极端环境/。


In [49]:
import jieba.posseg as pseg
words = pseg.lcut(string)
print(list(map(lambda x: list(x), words)))

[['在', 'p'], ['授课', 'v'], ['的', 'uj'], ['同时', 'c'], ['，', 'x'], ['这个', 'r'], ['“', 'x'], ['讲台', 'n'], ['”', 'x'], ['正以', 'd'], ['约', 'd'], ['23', 'm'], ['倍', 'm'], ['于', 'p'], ['音速', 'n'], ['的', 'uj'], ['第一', 'm'], ['宇宙速度', 'n'], ['环绕', 'v'], ['地球', 'n'], ['飞行', 'v'], ['，', 'x'], ['舱外', 's'], ['是', 'v'], ['真空', 'n'], ['、', 'x'], ['200', 'm'], ['多摄氏度', 'm'], ['的', 'uj'], ['温差', 'n'], ['、', 'x'], ['充满', 'a'], ['宇宙射线', 'l'], ['的', 'uj'], ['极端环境', 'x'], ['。', 'x']]


In [5]:
from snownlp import SnowNLP

string = '在授课的同时，这个“讲台”正以约23倍于音速的第一宇宙速度环绕地球飞行，舱外是真空、200多摄氏度的温差、充满宇宙射线的极端环境。'
s = SnowNLP(string)
result = s.words
print(len(result), '/'.join(result))

40 在/授课/的/同时/，/这个/“/讲台/”/正/以/约/23/倍于/音速/的/第一/宇宙/速度/环绕/地球/飞行/，/舱/外/是/真空/、200/多/摄氏度/的/温差/、/充满/宇宙/射线/的/极端/环境/。


In [9]:
print('Tags:', list(s.tags))
print('Sentiments:', s.sentiments)
print('Pinyin:', s.pinyin)


Tags: [('在', 'p'), ('授课', 'v'), ('的', 'u'), ('同时', 'n'), ('，', 'w'), ('这个', 'r'), ('“', 'w'), ('讲台', 'n'), ('”', 'w'), ('正', 'd'), ('以', 'p'), ('约', 'd'), ('23', 'o'), ('倍于', 'o'), ('音速', 'o'), ('的', 'u'), ('第一', 'm'), ('宇宙', 'n'), ('速度', 'n'), ('环绕', 'v'), ('地球', 'n'), ('飞行', 'vn'), ('，', 'w'), ('舱', 'n'), ('外', 'f'), ('是', 'v'), ('真空', 'n'), ('、200', 'Dg'), ('多', 'm'), ('摄氏度', 'q'), ('的', 'u'), ('温差', 'n'), ('、', 'w'), ('充满', 'v'), ('宇宙', 'n'), ('射线', 'k'), ('的', 'u'), ('极端', 'a'), ('环境', 'n'), ('。', 'w')]
Sentiments: 0.9998478687481432
Pinyin: ['zai', 'shou', 'ke', 'de', 'tong', 'shi', '，', 'zhe', 'ge', '“', 'jiang', 'tai', '”', 'zheng', 'yi', 'yue', '23', 'bei', '于', 'yin', 'su', 'de', 'di', 'yi', 'yu', 'zhou', 'su', 'du', 'huan', 'rao', 'di', 'qiu', 'fei', 'xing', '，', 'cang', 'wai', 'shi', 'zhen', 'kong', '、200', 'duo', 'she', 'shi', 'du', 'de', 'wen', 'cha', '、', 'chong', 'man', 'yu', 'zhou', 'she', 'xian', 'de', 'ji', 'duan', 'huan', 'jing', '。']


In [8]:
import thulac

string = '在授课的同时，这个“讲台”正以约23倍于音速的第一宇宙速度环绕地球飞行，舱外是真空、200多摄氏度的温差、充满宇宙射线的极端环境。'
t = thulac.thulac()
result = t.cut(string)
# print(result)
result = [i[0] for i in result]
print(len(result), '/'.join(result))

Model loaded succeed
41 在/授课/的/同时/，/这个/“/讲台/”/正/以/约/23/倍/于/音速/的/第一/宇宙/速度/环绕/地球/飞行/，/舱/外/是/真空/、/200/多/摄氏度/的/温差/、/充满/宇宙射线/的/极端/环境/。


In [7]:
import pynlpir

pynlpir.open()
string = '在授课的同时，这个“讲台”正以约23倍于音速的第一宇宙速度环绕地球飞行，舱外是真空、200多摄氏度的温差、充满宇宙射线的极端环境。'
result = pynlpir.segment(string)
# print(result)
result = [i[0] for i in result]
print(len(result), '/'.join(result))

41 在/授课/的/同时/，/这个/“/讲台/”/正/以/约/23/倍/于/音速/的/第一/宇宙/速度/环绕/地球/飞行/，/舱/外/是/真空/、/200/多/摄氏度/的/温差/、/充满/宇宙射线/的/极端/环境/。


In [3]:
from pyltp import Segmentor

string = '在授课的同时，这个“讲台”正以约23倍于音速的第一宇宙速度环绕地球飞行，舱外是真空、200多摄氏度的温差、充满宇宙射线的极端环境。'
segmentor = Segmentor("D:\download\edge\ltp_data_v3.4.0\cws.model")
result = list(segmentor.segment(string))
segmentor.release()
print(len(result), '/'.join(result))

41 在/授课/的/同时/，/这个/“/讲台/”/正/以/约/23/倍/于/音速/的/第一/宇宙/速度/环绕/地球/飞行/，/舱/外/是/真空/、/200/多/摄氏度/的/温差/、/充满/宇宙射线/的/极端/环境/。


In [2]:
for i in range(5):
    pass
print(i)

4
